<a href="https://colab.research.google.com/github/udayjinna/Basic_multimodel_rag/blob/main/Multimodal_rag_with_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multimodal RAG (PDF With Images)

![image.png](attachment:image.png)

In [ ]:
!pip install PyMuPDF
!pip install langchain_community
!pip install transformers
!pip install scikit-learn
!pip install faiss-cpu




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 33.5 MB/s eta 0:00:00


In [ ]:
#!pip uninstall -y google-generativeai google-ai-generativelanguage langchain-google-genai
!pip install google-generativeai langchain-google-genai

INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00


In [ ]:
#!pip install PyMuPDF
#!pip install langchain_community


#!pip uninstall -y google-generativeai google-ai-generativelanguage langchain-google-genai
#!pip install google-generativeai langchain-google-genai




import fitz  # PyMuPDF
from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage
from sklearn.metrics.pairwise import cosine_similarity
import os
import base64
import io
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [ ]:
###Clip Model


from google.colab import userdata

## set up the environment
os.environ["GOOGLE_API_KEY"]=userdata.get("GOOGLE_API_KEY")

### initialize the Clip Model for unified embeddings
clip_model=CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor=CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [ ]:
### Embedding functions
def embed_image(image_data):
    """Embed image using CLIP"""
    if isinstance(image_data, str):  # If path
        image = Image.open(image_data).convert("RGB")
    else:  # If PIL Image
        image = image_data

    inputs=clip_processor(images=image,return_tensors="pt")
    with torch.no_grad():
        features = clip_model.get_image_features(**inputs)
        # Normalize embeddings to unit vector
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

def embed_text(text):
    """Embed text using CLIP."""
    inputs = clip_processor(
        text=text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=77  # CLIP's max token length
    )
    with torch.no_grad():
        features = clip_model.get_text_features(**inputs)
        # Normalize embeddings
        features = features / features.norm(dim=-1, keepdim=True)
        return features.squeeze().numpy()

In [ ]:
## Process PDF
pdf_path="/content/Rag_pdf.pdf"
doc=fitz.open(pdf_path)
# Storage for all documents and embeddings
all_docs = []
all_embeddings = []
image_data_store = {}  # Store actual image data for LLM

# Text splitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)




In [ ]:
doc

Document('/content/Rag_pdf.pdf')

In [ ]:
for i,page in enumerate(doc):
    ## process text
    text=page.get_text()
    if text.strip():
        ##create temporary document for splitting
        temp_doc = Document(page_content=text, metadata={"page": i, "type": "text"})
        text_chunks = splitter.split_documents([temp_doc])

        #Embed each chunk using CLIP
        for chunk in text_chunks:
            embedding = embed_text(chunk.page_content)
            all_embeddings.append(embedding)
            all_docs.append(chunk)



    ## process images
    ##Three Important Actions:

    ##Convert PDF image to PIL format
    ##Store as base64 for gemini (which needs base64 images)
    ##Create CLIP embedding for retrieval

    for img_index, img in enumerate(page.get_images(full=True)):
        try:
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]

            # Convert to PIL Image
            pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

            # Create unique identifier
            image_id = f"page_{i}_img_{img_index}"

            # Store image as base64
            buffered = io.BytesIO()
            pil_image.save(buffered, format="PNG")
            img_base64 = base64.b64encode(buffered.getvalue()).decode()
            image_data_store[image_id] = img_base64

            # Embed image using CLIP
            embedding = embed_image(pil_image)
            all_embeddings.append(embedding)

            # Create document for image
            image_doc = Document(
                page_content=f"[Image: {image_id}]",
                metadata={"page": i, "type": "image", "image_id": image_id}
            )
            all_docs.append(image_doc)

        except Exception as e:
            print(f"Error processing image {img_index} on page {i}: {e}")
            continue

doc.close()


In [ ]:
all_docs

[Document(metadata={'page': 0, 'type': 'text'}, page_content='Retinexformer: One-stage Retinex-based\nTransformer for Low-light Image Enhancement\nYuanhao Cai 1, Hao Bian 1, Jing Lin 1,\nHaoqian Wang 1,* , Radu Timofte 2, Yulun Zhang 3,∗\n1 Tsinghua University, 2 University of W¨urzburg, 3 ETH Z¨urich\nAbstract\nWhen enhancing low-light images, many deep learning\nalgorithms are based on the Retinex theory. However, the\nRetinex model does not consider the corruptions hidden in\nthe dark or introduced by the light-up process. Besides,'),
 Document(metadata={'page': 0, 'type': 'text'}, page_content='the dark or introduced by the light-up process. Besides,\nthese methods usually require a tedious multi-stage training\npipeline and rely on convolutional neural networks, show-\ning limitations in capturing long-range dependencies. In\nthis paper, we formulate a simple yet principled One-stage\nRetinex-based Framework (ORF). ORF first estimates the\nillumination information to light up the 

In [ ]:
# Create unified FAISS vector store with CLIP embeddings
embeddings_array = np.array(all_embeddings)
embeddings_array

array([[ 0.00620995,  0.03214407, -0.03015697, ..., -0.11086666,
         0.0019284 , -0.00166416],
       [ 0.02514893, -0.0049546 , -0.0127898 , ..., -0.02670207,
         0.01263563,  0.00444957],
       [ 0.00423687, -0.00498299,  0.0095452 , ..., -0.00395005,
         0.01384856, -0.02543307],
       ...,
       [ 0.02513283,  0.00913819, -0.00731262, ..., -0.04079138,
         0.02180101,  0.01311531],
       [ 0.04029182,  0.02386706, -0.04083086, ..., -0.04506869,
         0.01502955,  0.02917191],
       [ 0.03149974,  0.02145283, -0.00527158, ..., -0.00457855,
        -0.0105234 ,  0.01146132]], dtype=float32)

In [ ]:
(all_docs,embeddings_array)

([Document(metadata={'page': 0, 'type': 'text'}, page_content='Retinexformer: One-stage Retinex-based\nTransformer for Low-light Image Enhancement\nYuanhao Cai 1, Hao Bian 1, Jing Lin 1,\nHaoqian Wang 1,* , Radu Timofte 2, Yulun Zhang 3,∗\n1 Tsinghua University, 2 University of W¨urzburg, 3 ETH Z¨urich\nAbstract\nWhen enhancing low-light images, many deep learning\nalgorithms are based on the Retinex theory. However, the\nRetinex model does not consider the corruptions hidden in\nthe dark or introduced by the light-up process. Besides,'),
  Document(metadata={'page': 0, 'type': 'text'}, page_content='the dark or introduced by the light-up process. Besides,\nthese methods usually require a tedious multi-stage training\npipeline and rely on convolutional neural networks, show-\ning limitations in capturing long-range dependencies. In\nthis paper, we formulate a simple yet principled One-stage\nRetinex-based Framework (ORF). ORF first estimates the\nillumination information to light up th

In [ ]:


# Create custom FAISS index since we have precomputed embeddings
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(all_docs, embeddings_array)],
    embedding=None,  # We're using precomputed embeddings
    metadatas=[doc.metadata for doc in all_docs]
)
vector_store

In [ ]:
import google.generativeai as genai

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(f"{m.name}")

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=userdata.get('GOOGLE_API_KEY'),
    temperature=0
)
llm

ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), temperature=0.0, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c3b4c168b30>, default_metadata=())

In [ ]:
def retrieve_multimodal(query, k=5):
    """Unified retrieval using CLIP embeddings for both text and images."""
    # Embed query using CLIP
    query_embedding = embed_text(query)

    # Search in unified vector store
    results = vector_store.similarity_search_by_vector(
        embedding=query_embedding,
        k=k
    )

    return results

In [ ]:
def multimodal_pdf_rag_pipeline(query):
    """Main pipeline for multimodal RAG."""
    # Retrieve relevant documents
    context_docs = retrieve_multimodal(query, k=5)

    # Create multimodal message
    message = create_multimodal_message(query, context_docs)

    # Get response from gemini
    response = llm.invoke([message])

    # Print retrieved context info
    print(f"\nRetrieved {len(context_docs)} documents:")
    for doc in context_docs:
        doc_type = doc.metadata.get("type", "unknown")
        page = doc.metadata.get("page", "?")
        if doc_type == "text":
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"  - Text from page {page}: {preview}")
        else:
            print(f"  - Image from page {page}")
    print("\n")

    return response.content

In [ ]:
if __name__ == "__main__":
    # Example queries
    queries = [
        "how many times the word ""low-light"" had come in this document"
    ]

    for query in queries:
        print(f"\nQuery: {query}")
        print("-" * 150)
        answer = multimodal_pdf_rag_pipeline(query)
        print(f"Answer: {answer}")
        print("=" * 150)


Query: how many times the word low-light had come in this document
------------------------------------------------------------------------------------------------------------------------------------------------------

Retrieved 5 documents:
  - Text from page 0: visibility and low contrast of low-light images and restore
the corruptions (e.g., noise, artifact, ...
  - Text from page 0: the dark or introduced by the light-up process. Besides,
these methods usually require a tedious mul...
  - Text from page 0: to-end. The training process is tedious and time-consuming.
This ICCV paper is the Open Access versi...
  - Text from page 0: decomposed into two components, i.e., reflectance and illu-
mination. Different from plain methods, ...
  - Text from page 0: The
second category is inspired by the Retinex theory. These
methods [54, 65, 66] usually suffer fro...


Answer: The word "low-light" appears 4 times in the provided document excerpts.
